## 7.1. 통계적 가설 검정
- 고전적인 가설검정에서는 어떤 기본적인 가설을 의미하는 귀무가설(H<sub>0</sub>, null hypothesis)과 비교하고 싶은 대립가설(H<sub>1</sub>, alternative hypothesis)로 구성되어 있으며, 통계를 사용해서 H<sub>0</sub>을 기각할지 말지를 결정한다.

## 7.2. 예시: 동전 던지기
- 동전에서 앞면이 나올 확률이 p라고 하면, 동전이 공평하다는 의미의 'p=0.5이다'는 귀무가설이 되고, 'p≠0.5이다'는 대립가설이 된다.
- 동전을 n번 던져서 앞면이 나온 횟수 X를 세는 것으로 검정을 진행해 보자.<br> 동전 던지기는 각각 베르누이 분포를 따를 것이며, 이는 X가 이항분포를 따르는 확률변수라는 것을 의미한다.
- 제1종 오류 :  비록 H<sub>0</sub>가 참이지만 H<sub>0</sub>를 기각하는 'false positive(가양성)' 오류
- 유의수준(significance) : 제1종 오류를 얼마나 허용해 줄 것인지를 의미, 보통 5%나 1%로 설정하는 경우가 많다.
- 제2종 오류 : H<sub>0</sub>가 거짓이지만 H<sub>0</sub>를 기각하지 않는 오류를 의미하기 때문에, 제2종 오류를 측정하기 위해서는 먼저 H<sub>0</sub>가 거짓이라는 것이 무엇을 의미하는지 알아볼 필요가 있다.
- 검정력(power) : 제2종 오류를 범하지 않을 확률

In [7]:
import math

def normal_approximation_to_binomial(n, p):
    '''Binomial(n, p)에 해당되는 mu(평균)와 sigma(표준편차) 계산'''
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [8]:
# ch06에서 사용한 정규분포의 누적분포함수
def normal_cdf(x, mu=0, sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

In [9]:
# 확률변수가 정규분포를 따른다는 가정하에, 
# normal_cdf를 사용하면 실제 동전 던지기로부터 얻은 값이 구간 안(혹은 밖)에 존재할 확률을 계산할 수 있다.

# 누적분포함수는 확률변수가 특정 값보다 작을 확률을 나타낸다.
normal_probability_below = normal_cdf

# 만약 확률변수가 특정 값보다 작지 않다면, 특정 값보다 크다는 것을 의미한다.
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# 만약 확률변수가 hi보다 작고 lo보다 작지 않으면, 확률변수는 hi와 lo 사이에 존재한다.
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# 만약 확률변수가 범위 밖에 존재한다면, 범위 안에 존재하지 않다는 것을 의미한다.
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [10]:
# 반대로, 확률이 주어졌을 때 평균을 중심으로 하는 (대칭적인) 구간을 구할 수도 있다.

def normal_upper_bound(probability, mu=0, sigma=1):
    '''P(Z <= z) = probability인 z값을 반환'''
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    '''P(Z >= z) = probability인 z값을 반환'''
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    '''입력한 probability 값을 포함하고, 평균을 중심으로 대칭적인 구간을 반환'''
    tail_probability = (1 - probability) / 2
    
    # 구간의 상항은 tail_probability 값 이상의 확률 값을 갖고 있다.
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # 구간의 하한은 tail_probability 값 이하의 확률 값을 갖고 있다.
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [11]:
# ch06에서 사용한 역함수
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    '''이진 검색을 사용해서 역함수를 근사'''
    # 표준정규분포가 아니라면 표준정규분포로 변환
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10.0, 0  # normal_cdf(-10)는 0에 근접
    hi_z, hi_p = 10.0, 1  # normal_cdf(10)는 1에 근접
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2  # 중간 값
        mid_p = normal_cdf(mid_z)  # 중간 값의 누적분포 값을 계산
        if mid_p < p:
            # 중간 값이 너무 작다면 더 큰 값들을 검색
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # 중간 값이 너무 크다면 더 작은 값들을 검색
            hi_z, hi_p = mid_z, mid_p
        else:
            break
    return mid_z

# 위의 함수는 원하는 확률 값에 가까워질 때까지 표준정규분포의 구간을 반복적으로 이등분한다.

In [12]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

In [13]:
# p가 0.5라고 가정할 때, 유의수준이 5%인 구간
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# p = 0.55인 경우의 실제 평균과 표준편차
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# 제2종 오류란 귀무가설(H_0)을 기각하지 못한다는 의미
# 즉, X가 주어진 구간 안에 존재할 경우를 의미
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
power

0.8865480012953671

In [14]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
hi  # 526 (<531, 분포 상위 부분에 더 높은 확률을 주기 위해서)

526.0073585242053

In [15]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
power 

0.9363794803307173

## 7.3. p-value
- 어떤 확률 값을 기준으로 구간을 선택하는 대신에, H<sub>0</sub>가 참이라고 가정하고 실제로 관측된 값보다 더 극단적인 값이 나올 확률을 구하는 것.

In [16]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # 만약 x가 평균보다 크다면, x보다 큰 부분이 꼬리다
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # 만약 x가 평균보다 작다면, x보다 작은 부분이 꼬리다
        return 2 * normal_probability_below(x, mu, sigma)
    
print('529.5:',two_sided_p_value(529.5, mu_0, sigma_0))  # 0.06207721579598835
print('530:', two_sided_p_value(530, mu_0, sigma_0)) # 0.05777957112359733
# 연속수정(continuity correction)때문에 530 대신 529.5가 더 정확하다.

529.5: 0.06207721579598835
530: 0.05777957112359733


In [18]:
import random

extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0  # 앞면이 나온 경우를 세어 본다.
                    for _ in range(1000))  # 동전을 1000번 던져서
    if num_heads >= 530 or num_heads <= 470:  # 그리고 극한값이
        extreme_value_count += 1  # 몇 번 나오는지 세어 본다.
        
print(extreme_value_count / 100000)

0.06204


In [19]:
# 계산된 p-value가 5%보다 크기 때문에 귀무가설을 기각하지 않는다.
# 만약 동전의 앞면이 532번 나왔다면 p-value는 5%보다 작을 것이고, 이 경우에는 귀무가설을 기각할 것이다.

two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

In [20]:
# 이전 가설검정에 비해 통계를 보는 관점만 다를 뿐 동일한 검정 방법이다.

upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [21]:
# 동전의 앞면이 525번 나왔다면 단측검정을 위한 p-value는 다음과 같이 계산되며, 기무가설을 기각하지 않을 것이다.
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582072

In [22]:
# 만약 동전의 앞면이 527번 나와다면, p-value는 다음과 같이 계산되며 기무가설을 기각할 것이다.
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

## 7.4. 신뢰구간
- 만약 사건에 대한 분포를 모른다면 관측된 값에 대한 신뢰구간(confidence interval)을 사용하여 가설을 검정할 수 있다.

In [ ]:
# 만약 p의 정확한 값을 알고 있다면 
# 중심극한정리를 사용하여 베르누이 확률변수들의 평균은 대략 평균이 p이고 
# 표준편차가 다음과 같은 정규분포로 추정할 수 있을 것이다.

math.sqrt(p * (1 - p) / 1000)

In [28]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print(sigma)
normal_two_sided_bounds(0.95, mu, sigma)

# 이 경우, 0.5는 신뢰구간 안에 있기 때문에, 동전은 공평하지 않다고 결론을 내릴 수 없다.

0.015791611697353755


(0.4940490278129096, 0.5559509721870904)

In [29]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print(sigma)
normal_two_sided_bounds(0.95, mu, sigma)

# '공평한 동전'에 대한 확률은 계산된 신뢰구간 밖에 존재하게 된다. 
# 공평한 동전이라는 가설은, 가설이 참이라면 모든 경우의 95%에 대해 참인 가설검정을 통과하지 못하게 된다.

0.015760710643876435


(0.5091095927295919, 0.5708904072704082)

## 7.5. p-value 해킹
- 귀무가설을 잘못 기각하는 경우가 5%인 가설검정은 정의에서 알 수 있듯이 모든 경우의 5%에서 귀무가설을 잘못 기각한다.

In [30]:
def run_experiment():
    '''동전을 1000번 던져서, True = 앞면, False = 뒷면'''
    return [random.random() < 0.5 
           for _ in range(1000)]

def reject_fairness(experiment):
    '''유의수준을 5%로 설정'''
    num_heads = len([flip 
                    for flip in experiment
                   if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() 
              for _ in range(1000)]
num_rejections = len([experiment
                     for experiment in experiments
                     if reject_fairness(experiment)])

print(num_rejections)

46


🌸데이터 과학을 제대로 해보고 싶다면 다음의 세 가지를 지키자!
- 첫째, 가설은 데이터를 보기 전에 새운다.
- 둘째, 데이터를 전처리할 때는 세워둔 가설을 잠시 잊는다.
- 셋째, p-value는 전부가 아니다.(대안으로 베이지안 추론을 사용할 수 있다.)

In [34]:
def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

In [35]:
def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [41]:
# 예를 들어, 1000명 중 200명이 A를 클릭했고, 1000명 중 180명이 B를 클릭했다고 보자.
# 그렇다면 통계치는 다음과 같다.

z = a_b_test_statistic(1000, 200, 1000, 180)
z

-1.1403464899034472

In [42]:
# 만약 두 분포의 평균이 같다면, 이렇게 큰 차이가 발생할 확률은 다음과 같다.

two_sided_p_value(z)

0.254141976542236

In [43]:
# 값이 크기 때문에 두 분포가 다르다고 결론을 내릴 수 없다.
# 만약 150명이 B를 클릭했다면 다음과 같은 결과가 나올 것이다.

z = a_b_test_statistic(1000, 200, 1000, 150)
print(z)
two_sided_p_value(z)

# 즉, A와 B가 동일하게 효과적이라면 이렇게 큰 차이가 발생할 확률은 불과 0.003이라는 것을 의미한다.

-2.948839123097944


0.003189699706216853

## 7.7. 베이지안 추론
- 알려지지 않은 파라미터를 확률변수로 보는 방법이 있다.
- 파라미터에 대한 사전분포가 주어지고, 관측된 데이터와 베이즈 정리를 사용하여 사후분포를 갱신할 수 있다.
- 통계적 검정에 대해 확률적으로 결론을 내는 대신에 파라미터에 대해 확률적으로 결론을 낼 수 있다.

In [44]:
def B(alpha, beta):
    '''모든 확률 값의 합이 1이 되도록 해주는 정규화 값'''
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

일반적으로 베타분포의 중심은 다음과 같고, <br>
alpha / (alpha + beta) <br>
alpha와 beta가 크면 클수록, 분포는 더 중심으로 몰려 있다.

## 7.8. 더 공부해 보고 싶다면
- Coursera의 Data Analysis and Statistical Inference 강의